In [1]:
import pandas as pd
import numpy as np

from nilearn.plotting import plot_roi, plot_stat_map
from nilearn.maskers import NiftiMasker
from nilearn import datasets
from pathlib import Path

In [21]:
def correct_mean_var(tseries):
    
# adapted from https://github.com/SIMEXP/niak/blob/master/commands/statistics/niak_correct_mean_var.m
# but for a 1D array
    
    nt = tseries.shape[0]
    tseries = tseries.reshape(nt,-1)
    
    mean_ts = np.mean(tseries,0)
    std_ts = tseries.std()
    
    tseries_n = (tseries - mean_ts)/std_ts
    tseries_n = np.ravel(tseries_n)
    
    return (tseries_n)

In [15]:
def mat2lvec(mat):
    
# from https://github.com/SIMEXP/niak/blob/29260775b77d45af540c73705c88cdffa7bbcc39/commands/formats/niak_mat2lvec.m

    N = mat.shape[1]
    mask = np.tril(np.ones(N))
    mask = mask>0
    vec = mat[mask]
    
    return (vec)

In [3]:
ts_path = Path("__file__").resolve().parents[1] / 'data' / 'sub-136S4836_ses-20120806_task-rest_atlas-mist64_desc-deconfound_timeseries.tsv'

In [20]:
# below code calculates maximal overlap between NIAK 64 and 12 regions
# adapted from https://github.com/pbellec/misc/blob/master/match_parcels.ipynb

masker = NiftiMasker(standardize=False, detrend=False)

In [5]:
mist_64 = datasets.fetch_atlas_basc_multiscale_2015()['scale064']
masker.fit(mist_64)
mist_vec_64 = masker.transform(mist_64)
np.unique(mist_vec_64)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
       40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52.,
       53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.],
      dtype=float32)

In [6]:
mist_12 = datasets.fetch_atlas_basc_multiscale_2015()['scale012']
masker.fit(mist_12)
mist_vec_12 = masker.transform(mist_12)
np.unique(mist_vec_12)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
      dtype=float32)

In [7]:
match = []
best_overlap = []
rois = []
for roi in np.unique(mist_vec_64):
    overlap = []
    for roi2 in np.unique(mist_vec_12):
        overlap.append(np.sum(mist_vec_12[mist_vec_64 == roi] == roi2) / np.sum(mist_vec_64 == roi))
    best_overlap.append(np.max(overlap))
    match.append(np.argmax(overlap)+1)
    #match.append(np.argmax(overlap))
    rois.append(roi)

In [8]:
#for ind, roi in enumerate(np.unique(mist_vec_64)):
    #print(f'roi {int(roi)} in MIST_64 has maximal overlap with MIST_12 {match[ind]} ({best_overlap[ind]})')

In [9]:
part = np.array(match)
part

array([ 1,  2,  3,  1, 12, 12, 11,  3,  3, 11, 12, 12, 10,  7, 12,  9,  1,
        2,  6,  8, 11,  3, 11,  4,  3, 11,  5,  9, 12,  5, 10, 11,  5,  8,
        5,  3,  9,  5, 10,  9,  3,  8,  5, 12,  7,  2, 11, 12, 10,  4, 11,
        8, 11, 10, 10,  7,  9,  6,  5, 11,  8,  7, 10,  4])

In [10]:
# below code to calculate afc, adapted from https://github.com/SIMEXP/niak/blob/master/commands/statistics/niak_build_afc.m

tseries = pd.read_csv(ts_path, sep='\t', header=0)
tseries

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,-2.165929,-5.442563,-4.111248,-8.392210,-2.031492,-4.869996,-6.041241,-2.472742,3.898415,-1.175191,...,2.145597,-1.359643,-0.346227,-3.733854,-1.384949,-3.909817,-1.817930,-1.254567,-5.964642,-4.373025
1,-2.257995,0.712717,-3.288544,5.992456,7.945017,8.219452,8.587873,-1.617604,7.808198,8.278348,...,-7.148190,13.555006,4.093032,5.529622,13.156146,14.597897,14.398783,7.980595,11.238144,7.836268
2,2.466664,9.139753,1.489788,6.490470,-2.829995,12.344437,9.325378,-5.998305,-5.198951,0.728305,...,-2.079952,6.373448,2.782069,7.152576,16.210598,8.555890,2.358696,0.483532,4.230913,5.525417
3,-3.768618,2.412955,-0.148263,5.879047,-4.234380,8.503470,5.206953,0.842916,-5.250636,-2.659013,...,9.275842,-2.916000,-1.432386,2.579397,-0.502385,1.133186,-1.752010,-1.905537,-1.796795,3.281056
4,0.959837,0.030847,1.063556,0.786120,-4.510458,5.963081,2.325958,-5.314883,-4.384069,0.912936,...,6.338491,1.459088,0.663091,0.032369,4.417964,0.580905,3.614512,-2.832625,4.945780,5.641926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2.812834,4.994494,5.263446,5.302578,2.124782,4.210033,2.928989,3.889542,-1.024371,0.844949,...,6.413964,-1.917948,1.643073,1.662484,-6.952022,-1.573185,6.132882,-4.178531,1.991206,-1.390446
136,-1.718192,-4.655519,2.062671,1.198132,-10.482201,4.897816,1.575583,0.767329,-13.192521,3.538400,...,-9.735510,6.274808,1.183247,8.253658,7.635677,2.574422,2.366360,1.327070,7.879750,8.650415
137,-1.228166,-6.344942,-8.175363,2.770522,7.382033,15.980279,-1.397408,-2.100754,-3.903297,3.063052,...,-13.092385,-3.249014,-5.034153,7.989132,-0.245305,-0.285986,-1.147499,-2.025469,5.280297,7.727571
138,-4.007438,0.927702,9.447940,-7.432448,-3.710224,5.276390,0.189880,-8.844813,-13.168867,-2.415726,...,-10.877328,-1.735155,-1.723741,-2.812652,9.059184,-0.649153,-10.431928,0.054155,1.036708,-1.793304


In [11]:
tseries = tseries.values
tseries[:,0]  #check matches expected matlab input

array([-2.16592932, -2.25799533,  2.4666636 , -3.76861807,  0.95983665,
        1.44530039,  0.4123176 , -2.17651617, -0.54852153, -2.49657688,
        2.69524661,  1.45094291,  2.46580188,  2.83097709,  2.78072097,
        3.789421  ,  1.86343248,  3.54253549, -1.69834722, -0.71918595,
       -6.49682154, -1.08137756,  0.67650634, -4.23207181, -2.93785406,
       -3.73412084, -0.9363547 ,  0.05775755, -1.56248069, -4.18776622,
       -3.79732643, -0.66219313,  1.30778611, -0.21609294, -0.10789552,
        1.59142761, -0.59211293,  1.11396123,  0.4473311 ,  2.25768614,
        2.86098541, -1.85709488, -4.88668642,  0.77042691,  3.75198029,
        2.02836121,  0.29309868, -0.61638084, -0.19100936,  0.96168983,
        2.25982481,  0.24564729, -2.67571727,  1.37671668,  2.28716243,
        5.22919987, -1.74887862, -5.61133388, -1.91102777, -0.19808274,
       -2.57821423,  1.29337943,  0.50029643,  3.28493065,  0.95254726,
       -0.87751195, -4.16928436, -3.8049263 ,  2.01817793,  6.48

In [12]:
nb_netwk = np.max(part)
afc = np.zeros([nb_netwk,nb_netwk])

In [14]:
# possibly to try and replace
#ts_corrected = [correct_mean_var(ts) for ts in tseries.T]
#ts_corrected = np.array(ts_corrected).T

In [16]:
# Computation of the mean signal per network
tseries_moy = np.zeros([tseries.shape[0], nb_netwk])  # for n rows x 12
siz_netwk = np.zeros([1,nb_netwk]) 

for num_n in range(1,nb_netwk+1):
    netwk_ts = []
    for ts, roi_netwk in zip(tseries.T, part):
        if roi_netwk == num_n:
            ts_n = correct_mean_var(ts)
            netwk_ts.append(ts_n)
    
    siz_netwk[:,num_n-1] = sum(part == num_n)
    tseries_moy[:,num_n-1] = np.mean(netwk_ts, axis=0)
    
afc = (1/(tseries_moy.shape[0]-1))*tseries_moy.T.dot(tseries_moy)
diagonal = np.diag_indices_from(afc)
afc[diagonal] = (afc[diagonal] - 1/siz_netwk)*siz_netwk/(siz_netwk-1)

In [23]:
afc_vec = mat2lvec(afc)
afc_vec

array([ 5.58531925e-01,  2.55429972e-01,  4.50130561e-01,  1.83965824e-01,
        9.51216442e-02,  3.08598245e-01,  3.52601934e-01,  2.90927523e-01,
        2.32672735e-01,  4.69771885e-01, -9.42042954e-02,  2.16464195e-01,
       -1.89728959e-01,  6.11227824e-02,  6.40652674e-01,  3.30743296e-01,
        3.42799152e-01,  1.22435643e-02,  4.07672730e-01,  1.27754699e-01,
        8.42002039e-01, -1.32847017e-01,  3.94830476e-01,  1.76884971e-02,
        2.18255909e-01,  4.44543831e-01,  1.49438027e-01,  7.79105376e-01,
        2.69285583e-01,  3.23276350e-01,  2.07085149e-01,  4.80815583e-01,
       -8.28252907e-04,  4.43805680e-01,  2.61397320e-01,  5.10865317e-01,
        1.47707799e-01,  1.99309150e-01,  1.42702934e-01,  2.62209946e-01,
        1.48305111e-01,  2.84700063e-01,  2.23707572e-01,  2.52864589e-01,
        2.06855277e-01,  2.71139013e-01,  3.55091014e-01,  1.37016722e-01,
        4.40811783e-01,  6.20069886e-02,  4.79091906e-01,  2.81398362e-01,
        5.07922595e-01,  

In [24]:
# issue: final result is slightly different from NIAK. afc is the same, but vectorized takes slightly different values
# tests below

(afc==afc.T).all() # checking afc array is symmetrical

True

In [39]:
N = afc.shape[1]
N

12

In [40]:
mask = np.tril(np.ones(N))
mask

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [41]:
mask = mask>0
mask

array([[ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True,  True, False, False, False, False, False, False, False,
        False, False, False],
       [ True,  True,  True, False, False, False, False, False, False,
        False, False, False],
       [ True,  True,  True,  True, False, False, False, False, False,
        False, False, False],
       [ True,  True,  True,  True,  True, False, False, False, False,
        False, False, False],
       [ True,  True,  True,  True,  True,  True, False, False, False,
        False, False, False],
       [ True,  True,  True,  True,  True,  True,  True, False, False,
        False, False, False],
       [ True,  True,  True,  True,  True,  True,  True,  True, False,
        False, False, False],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
        False, False, False],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False

In [42]:
vec = afc[mask]
vec

array([ 5.58531925e-01,  2.55429972e-01,  4.50130561e-01,  1.83965824e-01,
        9.51216442e-02,  3.08598245e-01,  3.52601934e-01,  2.90927523e-01,
        2.32672735e-01,  4.69771885e-01, -9.42042954e-02,  2.16464195e-01,
       -1.89728959e-01,  6.11227824e-02,  6.40652674e-01,  3.30743296e-01,
        3.42799152e-01,  1.22435643e-02,  4.07672730e-01,  1.27754699e-01,
        8.42002039e-01, -1.32847017e-01,  3.94830476e-01,  1.76884971e-02,
        2.18255909e-01,  4.44543831e-01,  1.49438027e-01,  7.79105376e-01,
        2.69285583e-01,  3.23276350e-01,  2.07085149e-01,  4.80815583e-01,
       -8.28252907e-04,  4.43805680e-01,  2.61397320e-01,  5.10865317e-01,
        1.47707799e-01,  1.99309150e-01,  1.42702934e-01,  2.62209946e-01,
        1.48305111e-01,  2.84700063e-01,  2.23707572e-01,  2.52864589e-01,
        2.06855277e-01,  2.71139013e-01,  3.55091014e-01,  1.37016722e-01,
        4.40811783e-01,  6.20069886e-02,  4.79091906e-01,  2.81398362e-01,
        5.07922595e-01,  

In [43]:
np.savetxt('output.csv', vec, delimiter=',', fmt='%f')